In [1]:
import tensorflow as tf
import numpy
import os
import json, pickle
import pandas
from functools import partial, reduce
import importlib

import sys
sys.path.append('../libs')

import flacdb
import prepare_data
import initialize
import data_pipeline
import loss_metrics
import conv_model
import plot_batch

In [2]:
importlib.reload(initialize)
sig_data, metadata = initialize.load_initial_data(must_have_bp=False)

In [3]:
save_path = '/scr1/mimic/initial_data/'
sig_data.to_pickle(save_path + 'sig_data.pkl')
metadata.to_pickle(save_path + 'metadata.pkl')

In [4]:
(metadata['chunk_count'] == 0).sum()

0

In [5]:
H = initialize.load_hypes()
parts = ['train', 'validation']
tensors, metadata, priors = initialize.run(H, parts=parts)

In [6]:
(metadata['chunk_count'] == 0).sum()

0

In [7]:
counts = tensors['train']['chunk_count']
counts = numpy.array(counts.flat_values)
counts.min()

1

In [8]:
dataset = {part: data_pipeline.build(H, tensors[part], part) for part in parts}
model = conv_model.build(H, priors)

In [9]:
model.fit(
    dataset['train'],
    validation_data = dataset['validation'],
    epochs = 2**20,
    steps_per_epoch = 2**H['steps_per_epoch_log2'],
    validation_steps = 2**H['validation_steps_log2'],
    validation_freq = range(0, 2**20, 2**H['validation_frequency_log2']),
    verbose = 1,
)

Train for 128 steps, validate for 4 steps
Epoch 1/1048576
128/128 [==============================] - 680s 5s/step - loss: 1.3162 - pressure_loss: 1.6076 - diagnosis_loss: 1.0248 - pressure_ABP_systolic: 25.9101 - pressure_ABP_diastolic: 13.3450 - pressure_ABP_pulse: 22.2522 - pressure_CVP_systolic: 7.6233 - pressure_CVP_diastolic: 4.7733 - pressure_CVP_pulse: 7.0250 - pressure_ICP_systolic: 3.6645 - pressure_ICP_diastolic: 3.8587 - pressure_ICP_pulse: 4.0255 - pressure_PAP_systolic: 9.7892 - pressure_PAP_diastolic: 5.1866 - pressure_PAP_pulse: 8.2797 - diagnosis_25000_diabetes_sensitivity: 0.4138 - diagnosis_25000_diabetes_specificity: 0.5409 - diagnosis_25000_diabetes_accuracy: 0.4773 - diagnosis_25000_diabetes_precise_sensitivity: 0.0548 - diagnosis_25000_diabetes_precise_threshold: 0.9811 - diagnosis_2720_hypercholesterolemia_sensitivity: 0.3043 - diagnosis_2720_hypercholesterolemia_specificity: 0.6549 - diagnosis_2720_hypercholesterolemia_accuracy: 0.4796 - diagnosis_2720_hyperchol

KeyboardInterrupt: 

In [7]:
ls /scr-ssd/mimic/waveforms/ | wc -l

379770


In [19]:
S = prepare_data.get_serialized()

In [9]:
import importlib

In [18]:
importlib.reload(prepare_data)

<module 'prepare_data' from '../libs/prepare_data.py'>

In [12]:
rec_segs = os.listdir(prepare_data.ROOT_SERIAL)

In [9]:
d = dataset['train']

In [11]:
counts = tensors['train']['chunk_count']
counts = numpy.array(counts.flat_values)
counts.min()

0

In [12]:
counts.max()

255

In [31]:
root = '/scr-ssd/mimic/waveforms/'
rec_seg_dirs = os.listdir(root)

In [32]:
empty_dirs = []
for i in rec_seg_dirs:
    if len(os.listdir(root + i)) == 0:
        empty_dirs.append(root + i)

In [33]:
len(empty_dirs)

0

In [13]:
counts.dtype

dtype('uint8')